In [72]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pandas as pd
from functools import reduce
sys.path.insert(0, "..")
from floral.utils.plotting import PLOTS_DIR
TABLES_DIR = os.path.join("..", "tables")
os.makedirs(TABLES_DIR, exist_ok=True)

In [73]:
# ================ CHOOSE EXPERIMENT BUNDLE ================ #
EXPERIMENT = "run_methods"
# ================ CHOOSE EXPERIMENT BUNDLE ================ #

In [74]:
AVAILABLE_EXPERIMENTS = [
    "run_methods",
    "run_methods_reduced",
    "run_methods_synthetic",
    "run_methods_general",
    "ab_floral",
    "ab_normlora",
    "hp_floral",
    "hp_floral_cifar100",
    "hp_convlora",
    "hp_batchnormlora_synthetic",
    "hp_batchnormlora_cifar100",
]

# ==================== DECLARE EXPERIMENT DATASETS, EXPERIMENT VARIABLES, AND DATASET METRICS ==================== #
# ----- Methods Performances ------ #
EXPERIMENT_SUFFIXES = ["_reduced", "_synthetic", "_general", "_cifar100", "_others"]
EXPERIMENT_DATASETS = {
    "run_methods": [
        "mnist_rotate",
        "mnist_label_shift",
        "cifar10_rotate",
        "cifar10_label_shift",
        "cifar100",
    ],
    "run_methods_reduced": [
        "mnist_rotate_reduced",
        "mnist_label_shift_reduced",
        "cifar10_rotate_reduced",
        "cifar10_label_shift_reduced",
        "cifar100_reduced",
    ],
    "run_methods_synthetic": [
        "synthetic_linear",
        "synthetic_mlp",
    ],
    "run_methods_general": [
        # "emnist",  # XXX
        # "shakespeare",  # XXX
        # "stackoverflow",  # XXX
    ],
    "ab_floral": [
        "mnist_rotate",
        "mnist_label_shift",
        "cifar10_rotate",
        "cifar10_label_shift",
    ],
    "ab_floral_cifar100": [
        "cifar100",
    ],
    "ab_normlora": [
        # "cifar100",  # XXX
        # "emnist",  # XXX
        # "stackoverflow",  # XXX
    ],
    "hp_floral": [
        "mnist_rotate",
        "mnist_label_shift",
        "cifar10_rotate",
        "cifar10_label_shift",
    ],
    "hp_floral_cifar100": [
        "cifar100",
    ],
    "hp_convlora": [
        "cifar10_rotate",
        "cifar10_label_shift",
        "cifar100",
        # "emnist",  # XXX
    ],
    "hp_batchnormlora_synthetic": [
        "synthetic_mlp_bn",
    ],
    "hp_batchnormlora_cifar100": [
        "cifar100_bn",
    ]
}

EXPERIMENT_VARIABLES = {
    "run_methods": ["method", "optimal_router"],
    "run_methods_reduced": ["method", "optimal_router"],
    "run_methods_synthetic": ["method", "optimal_router"],
    "run_methods_general": ["method"],
    "ab_floral": ["active_loras", "bias"],
    "ab_floral_cifar100": ["active_loras", "bias"],
    "hp_convlora": ["convlora_method"],
    "hp_floral": ["num_clusters", "rank"],
    "hp_floral_cifar100": ["num_clusters", "rank"],
    "hp_batchnormlora_synthetic": ["batchnorm_adaptor", "batchnorm_stats"],
    "hp_batchnormlora_cifar100": ["batchnorm_adaptor", "batchnorm_stats"],
}

EXPERIMENT_METRIC = {
    "run_methods": "acc_distributed",
    "run_methods_reduced": "acc_distributed",
    "run_methods_synthetic": "loss_distributed",
    "run_methods_general": None,  # lookup dataset metrics
    "ab_floral": "acc_distributed",
    "ab_floral_cifar100": "acc_distributed",
    "hp_convlora": "acc_distributed",
    "hp_floral": "acc_distributed",
    "hp_floral_cifar100": "acc_distributed",
    "hp_batchnormlora_synthetic": "loss_distributed",
    "hp_batchnormlora_cifar100": "acc_distributed",
}

DATASET_METRICS = {
    "synthetic_linear": ["loss_distributed"],
    "synthetic_mlp": ["loss_distributed"],
    "mnist_rotate":  ["acc_distributed"],
    "mnist_label_shift": ["acc_distributed"],
    "cifar10_rotate": ["acc_distributed"],
    "cifar10_label_shift": ["acc_distributed"],
    "cifar100": ["acc_distributed"],
    "mnist_rotate_reduced": ["acc_distributed"],
    "mnist_label_shift_reduced": ["acc_distributed"],
    "cifar10_rotate_reduced": ["acc_distributed"],
    "cifar10_label_shift_reduced": ["acc_distributed"],
    "cifar100_reduced": ["acc_distributed"],
    "synthetic_mlp_bn": ["loss_distributed"],
    "cifar100_bn": ["acc_distributed"],
    "emnist": ["acc_distributed"],
    "shakespeare": ["accuracy_top1_distributed", "accuracy_top5_distributed"],
    "stackoverflow": ["accuracy_top1_distributed", "accuracy_top3_distributed",
                      "accuracy_top5_distributed", "accuracy_top10_distributed"],
}


# ==================== DEFINE REPORT NAMES ==================== #
EXPERIMENT_TO_REPORT_NAME = {
    "run_methods": "Methods comparison",
    "run_methods_synthetic": "Methods comparison",
    "run_methods_general": "Methods comparison",
    "ab_floral": "FLoRAL adaptors ablation",
    "hp_convlora": "ConvLoRA types comparison",
    "hp_floral": "Number of Adaptors and their effective rank",
    "hp_batchnormlora": "Batch-Norm adaptors",
}

DATASET_TO_REPORT_NAME = {
    "synthetic_linear": "Synthetic Linear",
    "synthetic_mlp": "Synthetic MLP",
    "mnist_rotate":  "MNIST-Rotate",
    "mnist_label_shift": "MNIST-Label-Shift",
    "cifar10_rotate": "CIFAR-10-Rotate",
    "cifar10_label_shift": "CIFAR-10-Label-Shift",
    "cifar100": "CIFAR-100",
    "mnist_rotate_reduced": r"MNIST-Rotate(5\%)",
    "mnist_label_shift_reduced": r"MNIST-Label-Shift(5\%)",
    "cifar10_rotate_reduced": r"CIFAR-10-Rotate(5\%)",
    "cifar10_label_shift_reduced": r"CIFAR-10-Label-Shift(5\%)",
    "cifar100_reduced": r"CIFAR-100(5\%)",
    "synthetic_mlp_bn": "Synthetic MLP",
    "cifar100_bn": "CIFAR-100",
    "emnist": "FEMNIST",
    "shakespeare": "Shakespeare",
    "stackoverflow": "Stack Overflow",
}

VARIABLE_TO_REPORT_NAME = {
    "method": "Method",
    "optimal_router": "Optimal Router",
    "convlora_method": "ConvLoRA Type",
    "active_loras": "Active LoRAs",
    "bias": "Adaptive Bias",
    "num_clusters": r"$C$",
    "rank": r"$\rho$",
    "batchnorm_adaptor": "Adaptor",
    "batchnorm_stats": "Stats",
}

VARIABLE_VALUES_TO_REPORT_NAME = {
    "fedavg": "FedAvg",
    "floral": r"FLoRAL($\rho=1\%$)",
    "floral_optimalrouter": r"FLoRAL($\rho=1\%$)",
    "floral_10": r"FLoRAL($\rho=10\%$)",
    "floral_10_optimalrouter": r"FLoRAL($\rho=10\%$)",
    "locallora": "Local Adaptor",
    "ensemble": "Ensemble",
    "ensemble_optimalrouter": "Ensemble",

    "linear": "LoRA",
    "conv": "ConvLoRA",
    "linear+conv": "LoRA + ConvLoRA",

    "balanced": "Balanced",
    "balanced_2d": "Balanced 2D",
    "in": "In Layer",
    "out": "Out Layer",

    "regular": "Regular",
    "reparameterized": "Reparameterized",
    "local": "Local",
    "federated": "Federated",

    True: "Yes",
    False: "No",
    "none": "None",
}

METRIC_TO_REPORT_NAME = {
    "loss_distributed": "Loss",
    "loss_in_vocab_distributed": "Loss in Vocab",
    "acc_distributed": "Accuracy",
    "accuracy_distributed": "Accuracy",
    "accuracy_top1_distributed": "Accuracy",
    "accuracy_top3_distributed": "Accuracy (Top-3)",
    "accuracy_top5_distributed": "Accuracy (Top-5)",
    "accuracy_top3_distributed": "Accuracy (Top-10)",
}

In [75]:
assert EXPERIMENT in AVAILABLE_EXPERIMENTS

df_list = []
for dataset in EXPERIMENT_DATASETS[EXPERIMENT]:
    experiment_base = EXPERIMENT
    for suffix in EXPERIMENT_SUFFIXES:
        experiment_base = experiment_base.removesuffix(suffix)
    experiment_name = f"{experiment_base}_{dataset}"
    metrics_file = os.path.join("..", PLOTS_DIR, experiment_name, "metrics.csv")
    if not os.path.exists(metrics_file):
        print(f"metrics csv for experiment '{experiment_name}' does not exist! File: {metrics_file}")
        continue
    metrics_df = pd.read_csv(metrics_file)  # TODO: might be outdated, get from history.pkl directly
    variables_df = metrics_df[EXPERIMENT_VARIABLES[EXPERIMENT]]
    variables_df = variables_df.apply(
        lambda col: col.apply(lambda val: VARIABLE_VALUES_TO_REPORT_NAME.get(val, val))
    )
    variables_df[DATASET_TO_REPORT_NAME[dataset]] = metrics_df[EXPERIMENT_METRIC[EXPERIMENT]]
    df_list.append(variables_df)

experiment_metrics_df = reduce(
    lambda df1, df2: pd.merge(df1, df2, how="outer", on=EXPERIMENT_VARIABLES[EXPERIMENT]), df_list)
experiment_metrics_df = experiment_metrics_df.sort_values(by=EXPERIMENT_VARIABLES[EXPERIMENT])
# experiment_metrics_df = experiment_metrics_df.groupby(by=EXPERIMENT_VARIABLES[EXPERIMENT]).mean()
experiment_metrics_df = experiment_metrics_df.rename(VARIABLE_TO_REPORT_NAME, axis="columns")
experiment_metrics_df

,Method,Optimal Router,MNIST-Rotate,MNIST-Label-Shift,CIFAR-10-Rotate,CIFAR-10-Label-Shift,CIFAR-100
0,Ensemble,No,91.891892,93.593593,73.290000,56.710000,35.174
1,Ensemble,Yes,94.994993,95.295295,73.830000,72.720000,77.062
2,FLoRAL($\rho=10\%$),No,93.093092,94.494494,72.470001,57.490000,57.350
3,FLoRAL($\rho=10\%$),Yes,95.195194,94.794794,72.689999,75.730000,NaN
4,FLoRAL($\rho=1\%$),No,91.691691,93.193193,70.210000,74.150000,51.716
5,FLoRAL($\rho=1\%$),Yes,92.892893,92.992993,71.040000,73.069999,50.280
6,FedAvg,No,90.990991,24.924925,64.400001,22.070000,12.000
7,Local Adaptor,No,86.486487,84.684685,66.040000,69.080000,52.600


In [76]:
float_format = "%.2f" if "acc" in EXPERIMENT_METRIC[EXPERIMENT] else "%.4f"
experiment_metrics_df.to_latex(os.path.join(TABLES_DIR, EXPERIMENT + ".tex"), float_format=float_format)